In [18]:
def maj(x,y,z):
    return (((x) & (y)) ^ ((x) & (z)) ^ ((y) & (z)))

def ch(x,y,z):
    return (((x) & (y)) ^ (((x) ^ 1) & (z)))

def KSG128(state):
    return (state[12] ^ state[154] ^ maj(state[235], state[61], state[193]))

def FBK128(state, ks, ca, cb):
    ks = KSG128(state)
    f  = (state[0] ^ (state[107] ^ 1) ^ maj(state[244], state[23], state[160])^ 
          ch(state[230], state[111], state[66]) ^ (ca & state[196]) ^ (cb & (ks)))    
    return ks, f

def Encrypt_StateUpdate128_1bit(state, plaintextbit, ks, ca, cb):
    state[289] ^= state[235] ^ state[230]
    state[230] ^= state[196] ^ state[193]
    state[193] ^= state[160] ^ state[154]
    state[154] ^= state[111] ^ state[107]
    state[107] ^= state[66]  ^ state[61]
    state[61]  ^= state[23]  ^ state[0]

    ks, f  = FBK128(state, ks, ca, cb)
    
    for j in range(292):
        state[j] = state[j+1]
        
    state[292] = f ^ plaintextbit
    
    ciphertextbit = ks ^ plaintextbit
    return state, ks, ciphertextbit


def Decrypt_StateUpdate128_1bit(state, plaintextbit, ciphertextbit, ks, ca, cb):
    state[289] ^= state[235] ^ state[230]
    state[230] ^= state[196] ^ state[193]
    state[193] ^= state[160] ^ state[154]
    state[154] ^= state[111] ^ state[107]
    state[107] ^= state[66]  ^ state[61]
    state[61]  ^= state[23]  ^ state[0]

    ks, f = FBK128(state, ks, ca, cb)

    for j in range(291):
        state[j] = state[j+1]
        
    plaintextbit = ks ^ ciphertextbit
    state[292] = f ^ plaintextbit
    return state, ks, plaintextbit


def acorn128_enc_onebyte(state, plaintextbyte, cabyte, cbbyte):
    ciphertextbyte = 0
    kstem = 0
    ksbyte = 0
    for i in range(8):
        ca = (cabyte >> i) & 1
        cb = (cbbyte >> i) & 1
        plaintextbit = (plaintextbyte >> i) & 1
        #print(plaintextbit)
        state, kstem, ciphertextbit = Encrypt_StateUpdate128_1bit(state, plaintextbit, kstem, ca, cb)
        ciphertextbyte |= (ciphertextbit << i)
        ksbyte |= (kstem << i)
    return state, ksbyte, ciphertextbyte

def acorn128_dec_onebyte(state, plaintextbyte, ciphertextbyte, cabyte, cbbyte):
    plaintextbyte = 0
    kstem = 0
    for i in range(8):
        ca = (cabyte >> i) & 1
        cb = (cbbyte >> i) & 1
        plaintextbyte = (plaintextbyte >> i) & 1;
        state, kstem, plaintextbit = Decrypt_StateUpdate128_1bit(state, plaintextbit, ciphertextbit, kstem, ca, cb)
        plaintextbyte |= (plaintextbit << i)
    return state, ksbyte, plaintextbyte


def acorn128_initialization(key, iv):
    #unsigned char m[293], ks, tem;
    #initialize the state to 0
    state = [0 for i in range(294)]
    
    #set the value of m
    m = key + iv + [key[i & 0xf] for i in range(32, 224)]    
    m[32] ^= 1
    m += [0]*(293-len(m)+1)

    #run the cipher for 1792 steps    
    #print(len(m))
    #for j,k in enumerate(m):
    #    #print(k, " ", end="")
    #    if not (j+1) % 16:
    #        #print()
    #print()
    for i in range(224):
        #print(m[i])
        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, m[i], 0xff, 0xff)
    return state


#the finalization state of acorn
def acorn128_tag_generation(msglen, adlen, maclen, mac, state):
    plaintextbyte  = 0
    ciphertextbyte = 0
    ksbyte = 0
    for i in range(int(768/8)):
        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, plaintextbyte, 0xff, 0xff)
        if i >= (768/8 - 16):
            mac[i-(int(768/8)-16)] = ksbyte
    return mac


#encrypt a message.
def crypto_aead_encrypt(c, m, mlen, ad, adlen, nsec, npub, k):
    #plaintextbyte, ciphertextbyte, ksbyte, mac[16];
    #unsigned char state[293];
    #unsigned char ca, cb;
    mac = [0]*16

    #initialization stage
    state = acorn128_initialization(k, npub)
    print("\n\nacorn128_initialization: ");
    for j,k in enumerate(state):
        print(k, " ", end="")
        if not (j+1) % 16:
            print()
    print()

    #process the associated data
    for i in range(adlen):
        #print(ad[i])
        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, ad[i], 0xff, 0xff)
        
    '''print("\n\nassociated data: ");
    for j,k in enumerate(state):
        print(k, " ", end="")
        if not (j+1) % 16:
            print()
    print()'''

    for i in range(int(256/8)):
        if i == 0:
            plaintextbyte = 0x1
        else:
            plaintextbyte = 0
        if i < 128/8:
            ca = 0xff
        else:
            ca = 0
        cb = 0xff
        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, plaintextbyte, ca, cb)
 
    '''print("\n\nacorn128_enc_onebyte: ");
    for j,k in enumerate(state):
        print(k, " ", end="")
        if not (j+1) % 16:
            print()
    print()'''
    
    #process the plaintext
    for i in range(mlen):
        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, m[i], 0xff, 0 )
        #print(ciphertextbyte)
        c[i] = ciphertextbyte
    for i in range(int(256/8)):
        if i == 0:
            plaintextbyte = 0x1
        else:
            plaintextbyte = 0

        if ( i < 128/8):
            ca = 0xff
        else:
            ca = 0

        cb = 0

        state, ksbyte, ciphertextbyte = acorn128_enc_onebyte(state, plaintextbyte, ca, cb)

    #finalization stage, we assume that the tag length is a multiple of bytes
    mac = acorn128_tag_generation(mlen, adlen, 16, mac, state)
    clen = mlen + 16
    for i in mac:
        print(hex(i))
    c = c[0:mlen] + mac
    #memcpy(c+mlen, mac, 16);
    return clen, c

In [19]:
key = [0 for i in range(16)]
key[0] = 1;
iv = [0 for i in range(16)]
plaintext  = [i%256 for i in range(4096)]
ciphertext = [0     for i in range(4096)]
ad         = [i%7   for i in range(4096)]
msglen     = 1003
adlen      = 1003

print('key       :', key)
print('iv        :', iv)
print('plaintext :', plaintext)
#print('ciphertext:', ciphertext)
#print('ad        :', ad)

key       : [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
iv        : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
plaintext : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 

In [20]:
clen, ciphertext = crypto_aead_encrypt(ciphertext, plaintext, msglen, ad, adlen, 0, iv, key)

print("\n\ciphertext: ");
for j,k in enumerate(ciphertext):
    print(hex(k), " ", end="")
    if not (j+1) % 16:
        print()
print()        



acorn128_initialization: 
1  0  1  0  1  0  0  0  1  1  0  0  1  0  0  1  
0  1  1  0  1  0  0  1  1  1  1  1  0  1  0  0  
1  0  1  0  1  1  1  0  1  0  1  0  0  0  0  1  
0  1  0  1  0  1  1  0  1  0  0  1  1  1  0  0  
1  0  1  0  1  0  1  1  0  0  1  1  1  0  0  1  
1  0  0  0  0  0  0  1  1  0  0  1  1  1  0  0  
1  1  1  0  1  0  0  1  1  1  0  0  0  0  0  0  
1  0  1  1  0  0  0  1  1  0  1  1  1  1  1  0  
1  0  1  1  0  0  1  0  1  0  0  1  0  0  1  0  
0  0  0  0  0  0  0  0  0  1  1  0  0  1  0  1  
1  1  1  1  1  0  0  0  1  0  1  0  0  0  1  0  
1  1  0  1  0  0  1  1  1  0  1  1  0  0  1  1  
0  1  0  0  0  1  1  1  1  0  0  0  1  0  0  0  
0  0  1  0  0  0  1  0  0  0  0  0  1  1  0  0  
0  1  0  0  1  1  1  1  1  1  1  0  1  0  1  0  
1  0  0  1  1  1  0  1  0  1  0  1  0  0  0  1  
0  0  1  1  0  0  1  0  0  1  0  1  1  1  0  1  
1  1  0  1  0  0  1  0  0  1  1  0  0  0  0  1  
0  0  0  1  1  0  
0x88
0x26
0x48
0x62
0x24
0xaa
0x10
0xea
0xd7
0x1e
0x77
0xc
0xed
0xa6
0x

In [42]:
print(key)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
